##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CycleGAN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/cyclegan"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Это руководство демонстрирует непарное преобразование изображений в изображения с использованием GAN, как описано в [Непарном преобразовании изображения с использованием циклически согласованных состязательных сетей](https://arxiv.org/abs/1703.10593), также известном как `CycleGAN`. В документе предлагается метод, который может взять характеристики одной области изображения и выяснить, как эти характеристики могут быть переведены в другую область изображения, и все это в отсутствие каких-либо парных обучающих примеров.

В этом руководстве предполагается, что вы знакомы с `Pix2Pix`, о которой вы можете узнать из [учебного пособия по Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix). Код `CycleGAN` похож на `Pix2Pix`, основное отличие - дополнительная функция потерь и использование непарных обучающих данных.

`CycleGAN` использует потерю согласованности цикла, чтобы обеспечить обучение без использования парных данных. Другими словами, он может переводить из одного домена в другой без взаимно однозначного сопоставления между исходным и целевым доменами.

Это позволяет выполнять множество интересных задач, таких как улучшение фотографий, раскрашивание изображений, перенос стилей и т.д. Все, что вам нужно, это исходный и целевой набор данных(который представляет собой просто каталог изображений).

![Выходное изображение 1](images/horse2zebra_1.png)
![Выходное изображение 2](images/horse2zebra_2.png)

## Настройка входного конвейера

Установите пакет [tensorflow_examples](https://github.com/tensorflow/examples), который позволяет импортировать генератор и дискриминатор.

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.experimental.AUTOTUNE

## Входной конвейер

В этом руководстве мы будем обучать модель переводить изображения лошадей в изображения зебр. Вы можете найти этот и аналогичные наборы данных [здесь](https://www.tensorflow.org/datasets/datasets#cycle_gan).

Как упоминалось в [статье](https://arxiv.org/abs/1703.10593), примените случайный шум и зеркальное отображение к набору обучающих данных. Это некоторые из методов аугментации изображений, которые позволяют избежать переобучения.

Подобное было сделано в [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#load_the_dataset)

* При случайном шуме изображение изменяется до `286 x 286`, а затем случайным образом обрезается до `256 x 256`.
* При зеркальном отображении изображение переворачивается по горизонтали, т.е. слева направо

In [ ]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
# нормализация изображения до [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [ ]:
def random_jitter(image):
  # изменение размера до 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # обрезка изображения до 256 x 256 x 3
  image = random_crop(image)

  # зеркальное отображение
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image, label):
  image = normalize(image)
  return image

In [ ]:
train_horses = train_horses.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(1)

train_zebras = train_zebras.map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(1)

test_horses = test_horses.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(1)

test_zebras = test_zebras.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(1)

In [ ]:
sample_horse = next(iter(train_horses))
sample_zebra = next(iter(train_zebras))

In [ ]:
plt.subplot(121)
plt.title('Horse')
plt.imshow(sample_horse[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Horse with random jitter')
plt.imshow(random_jitter(sample_horse[0]) * 0.5 + 0.5)

In [ ]:
plt.subplot(121)
plt.title('Zebra')
plt.imshow(sample_zebra[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Zebra with random jitter')
plt.imshow(random_jitter(sample_zebra[0]) * 0.5 + 0.5)

## Импорт и повторное использование модели Pix2Pix

Импортируйте генератор и дискриминатор, используемые в [Pix2Pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py) через установленный [tensorflow_examples](https://github.com/tensorflow/examples) пакет.

Архитектура модели, используемая в этом руководстве, очень похожа на ту, что использовалась в [pix2pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). Но есть некоторые отличия:

* Cyclegan использует [нормализацию экземпляра](https://arxiv.org/abs/1607.08022) вместо [пакетной нормализации](https://arxiv.org/abs/1502.03167).
* В [CycleGAN paper](https://arxiv.org/abs/1703.10593) используется модифицированный генератор на основе `resnet`. В этом руководстве для простоты используется модифицированный генератор ʻunet`.

Здесь обучаются 2 генератора (G и F) и 2 дискриминатора (X и Y).

* Генератор `G` учится преобразовывать изображение `X` в изображение `Y` $(G: X -> Y)$
* Генератор `F` учится преобразовывать изображение` Y` в изображение `X`. $(F: Y -> X)$
* Дискриминатор `D_X` учится различать изображение` X` и сгенерированное изображение `X` (`F(Y)`).
* Дискриминатор `D_Y` учится различать изображение` Y` и сгенерированное изображение `Y` (`G(X)`).

![Модель Cyclegan](images/cyclegan_model.png)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
to_zebra = generator_g(sample_horse)
to_horse = generator_f(sample_zebra)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real zebra?')
plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real horse?')
plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap='RdBu_r')

plt.show()

## Функция потерь

В CycleGAN нет парных данных для обучения, поэтому нет гарантии, что входная пара `x` и целевая пара `y` связаны во время обучения. Таким образом, чтобы обеспечить обучение сети правильному отображению, авторы предлагают потерю согласованности цикла.

Потери дискриминатора и потери генератора аналогичны тем, которые используются в [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#define_the_loss_functions_and_the_optimizer).

In [ ]:
LAMBDA = 10

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

Согласованность цикла означает, что результат должен быть близок к исходному вводу. Например, если кто-то переводит предложение с английского на французский, а затем переводит его обратно с французского на английский, то полученное предложение должно быть таким же, как исходное.

Расчет величины потерь согласованности цикла:

* Изображение $X$ передается через генератор $G$, который отдает сгенерированное изображение $\hat{Y}$.
* Сгенерированное изображение $\hat{Y}$ передается через генератор $F$, который возвращает зацикленное изображение $\hat{X}$.
* Средняя абсолютная ошибка вычисляется между $X$ и $\hat{X}$.

$$forward\ cycle\ consistency\ loss: X -> G(X) -> F(G(X)) \sim \hat{X}$$

$$backward\ cycle\ consistency\ loss: Y -> F(Y) -> G(F(Y)) \sim \hat{Y}$$


![Потери цикла](images/cycle_loss.png)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

Как показано выше, генератор $G$ отвечает за преобразование изображения $X$ в изображение $Y$. Потеря идентичности говорит о том, что если вы загрузили изображение $Y$ в генератор $G$, оно должно дать реальное изображение $Y$ или что-то близкое к изображению $Y$.

$$Identity\ loss = |G(Y) - Y| + |F(X) - X|$$

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

Инициализируйте оптимизаторы для всех генераторов и дискриминаторов.

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Чекпойнты

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# если чекпойнт существует - восстанавливаем последний
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Обучение
 
Примечание. Этот пример модели обучен на меньшем количестве эпох(40), чем описанный в документе(200), чтобы сократить время обучения. Поэтому прогнозы могут быть менее точными.

In [ ]:
EPOCHS = 40

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

Несмотря на то, что цикл обучения выглядит сложным, он состоит из четырех основных шагов:

* Получаем прогнозы.
* Рассчитываем потери.
* Рассчитываем градиенты с помощью обратного распространения ошибки.
* Применяем градиенты к оптимизатору.

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent установлен в True, потому что GradientTape используется 
  # более одного раза для вычисления градиентов.
  with tf.GradientTape(persistent=True) as tape:
    # Генератор G переводит X -> Y
    # Генератор F переводит Y -> X.
    
    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x и same_y используются для определения потерь.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # расчитываем потери
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Общие потери генератора = состязательные потери + потери цикла
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Расчитываем градиент для генератора и дискриминатора
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Применяем градиенты к оптимайзеру
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n+=1

  clear_output(wait=True)
  # Используем согласованное изображение(sample_horse), 
  # чтобы было ясно видно, как работает модель.
  generate_images(generator_g, sample_horse)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Сохраняем чекпойнт для эпохи {} в {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Время затраченное на эпоху {} составило {} сек\n'.format(epoch + 1,
                                                      time.time()-start))

## Генерация с использованием тестового набора данных

In [ ]:
# Запускаем обученную модель с тестовым датасетом
for inp in test_horses.take(5):
  generate_images(generator_g, inp)

## Следующие шаги

В этом руководстве показано, как реализовать CycleGAN, начиная с генератора и дискриминатора, реализованного в руководстве [Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix). В качестве следующего шага вы можете попробовать использовать другой набор данных из [TensorFlow Datasets](https://www.tensorflow.org/datasets/datasets#cycle_gan).

Вы также можете обучить модель с бОльшим количеством эпох, чтобы улучшить результаты, или можете реализовать модифицированный генератор ResNet, используемый в [статье](https://arxiv.org/abs/1703.10593), вместо используемого генератора U-Net.